In [2]:
# -*- coding: utf-8 -*-


import websocket
import os
import json
import threading
import time
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame


API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")

endpoint = "https://paper-api.alpaca.markets"

headers = {'APCA-API-KEY-ID': API_KEY,
           'APCA-API-SECRET-KEY': API_SECRET}
api = tradeapi.REST(headers["APCA-API-KEY-ID"], headers["APCA-API-SECRET-KEY"], base_url='https://paper-api.alpaca.markets')
#tickers = []
tickers = []


ltp = {} #dictionary to store ltp information for each ticker
prev_close = {} #dictionary to store previous day's close price information for each ticker 
perc_change = {} #dictionary to store percentage change from yesterday's close for each ticker
traded_tickers = [] #storing tickers which have been traded and therefore to be excluded
max_pos = 5000 #max position size for each ticker

def hist_data(symbols, start_date ="2022-02-01", timeframe="Minute"):
    """
    returns historical bar data for a list of tickers e.g. symbols = ["MSFT,AMZN,GOOG"]
    """
    df_data = {}
    api = REST(headers["APCA-API-KEY-ID"], headers["APCA-API-SECRET-KEY"], base_url=endpoint)
    for ticker in symbols:
        if timeframe == "Minute":
            df_data[ticker] = api.get_bars(ticker, TimeFrame.Minute, start_date, adjustment='all').df
        elif timeframe == "Hour":
            df_data[ticker] = api.get_bars(ticker, TimeFrame.Hour, start_date, adjustment='all').df
        else:
            df_data[ticker] = api.get_bars(ticker, TimeFrame.Day, start_date, adjustment='all').df
    return df_data

    
data_dump = hist_data(tickers, timeframe="Day")

#initializing the dictionaries
for ticker in tickers:
    prev_close[ticker] = data_dump[ticker]["close"][-2]
    ltp[ticker] = data_dump[ticker]["close"][-1]
    perc_change[ticker] = 0

def on_open(ws):
    auth = {"action": "auth", "key": headers["APCA-API-KEY-ID"], "secret": headers["APCA-API-SECRET-KEY"]}
    
    ws.send(json.dumps(auth))
    
    message = {"action":"subscribe","trades":tickers}
                
    ws.send(json.dumps(message))
 
def on_message(ws, message):
    print(message)
    tick = json.loads(message)
    tkr = tick[0]["S"]
    ltp[tkr] = float(tick[0]["p"])
    perc_change[tkr] = round((ltp[tkr]/prev_close[tkr] - 1)*100,2)   
    

def connect():
    ws = websocket.WebSocketApp("wss://stream.data.alpaca.markets/v2/iex", on_open=on_open, on_message=on_message)
    ws.run_forever()

def pos_size(ticker):
    return max(1,int(max_pos/ltp[ticker]))

def signal(traded_tickers):
    #print(traded_tickers)
    for ticker, pc in perc_change.items():
        (ticker, pc)
        if pc > .5 and ticker not in traded_tickers:
            api.submit_order(ticker, pos_size(ticker), "buy", "market", "ioc")
            time.sleep(2)
            try:
                filled_qty = api.get_position(ticker).qty
                time.sleep(1)
            #    api.submit_order(ticker, int(filled_qty), "sell", "trailing_stop", "day", trail_percent = "3")
             #   traded_tickers.append(ticker)
            except Exception as e:
                print(ticker, e)
        if pc < -.5 and ticker not in traded_tickers:
            api.submit_order(ticker, pos_size(ticker), "sell", "market", "ioc")
            time.sleep(2)
            try:
                filled_qty = api.get_position(ticker).qty
                time.sleep(1)
            #    api.submit_order(ticker, -1*int(filled_qty), "buy", "trailing_stop", "day", trail_percent = "3")
             #   traded_tickers.append(ticker)
            except Exception as e:
                print(ticker, e)

con_thread = threading.Thread(target=connect, daemon=True)
con_thread.start()

starttime = time.time()
timeout = starttime + 60*60
while time.time() <= timeout:
    for ticker in tickers:
        print("percent change for {} = {}".format(ticker,perc_change[ticker]))
        signal(traded_tickers)
    time.sleep(60 - ((time.time() - starttime) % 60))

#closing all positions and cancelling all orders at the end of the strategy  
api.close_all_positions(
    
)
api.cancel_all_orders()
time.sleep(5)


percent change for TQQQ = 0
[{"T":"success","msg":"connected"}]
[{"T":"success","msg":"authenticated"}]
[{"T":"subscription","trades":["TQQQ"],"quotes":[],"bars":[],"updatedBars":[],"dailyBars":[],"statuses":[],"lulds":[],"corrections":["TQQQ"],"cancelErrors":["TQQQ"]}]
[{"T":"t","S":"TQQQ","i":2687,"x":"V","p":22.37,"s":79,"c":["@","I"],"z":"C","t":"2022-10-04T17:01:17.036344457Z"}]
[{"T":"t","S":"TQQQ","i":2688,"x":"V","p":22.32,"s":362,"c":["@"],"z":"C","t":"2022-10-04T17:01:40.317138636Z"},{"T":"t","S":"TQQQ","i":2689,"x":"V","p":22.32,"s":100,"c":["@"],"z":"C","t":"2022-10-04T17:01:40.317138636Z"}]
[{"T":"t","S":"TQQQ","i":2690,"x":"V","p":22.3,"s":300,"c":["@"],"z":"C","t":"2022-10-04T17:02:01.706953505Z"}]
[{"T":"t","S":"TQQQ","i":2691,"x":"V","p":22.3,"s":695,"c":["@"],"z":"C","t":"2022-10-04T17:02:01.707212155Z"}]
[{"T":"t","S":"TQQQ","i":2692,"x":"V","p":22.3,"s":1705,"c":["@"],"z":"C","t":"2022-10-04T17:02:01.707325654Z"},{"T":"t","S":"TQQQ","i":2693,"x":"V","p":22.3,"s":100